In [3]:
from torcheval.metrics.text import Perplexity
import torch

In [34]:
metric = Perplexity()

logits = torch.tensor([[[0.1, 0.9]]])
labels = torch.tensor([[0]])

metric.update(logits, labels)

metric.compute()
logits = torch.tensor([[[0.3, 0.6]]])
labels = torch.tensor([[1]])

metric.update(logits, labels)
metric.compute()


tensor(2.3696, dtype=torch.float64)

In [26]:
import numpy as np
## perplexity = exp(sum -log(p_i) / N)

perplexity = np.exp
perplexity

0.830029230684137

In [3]:
import os
import numpy as np
data_type = 'instruct'

block_size = 2048
dataset = 'instruct2'
tokens_per_iter = block_size*32
data_dir = os.path.join(os.path.dirname(os.path.realpath("__file__")), 'data', dataset)
train_data = np.memmap(os.path.join(data_dir, 'train.bin'), dtype=np.uint16, mode='r')
val_data = np.memmap(os.path.join(data_dir, 'val.bin'), dtype=np.uint16, mode='r')

print("Iterations per epoch:", train_data.shape[0] // tokens_per_iter)


if data_type == 'instruct':
    mask_train = np.memmap(os.path.join(data_dir, 'inp_shape_train.bin'), dtype=np.uint16, mode='r')
    mask_val = np.memmap(os.path.join(data_dir, 'inp_shape_val.bin'), dtype=np.uint16, mode='r')
    train_data = train_data.reshape(-1, block_size)
    val_data = val_data.reshape(-1, block_size)
    mask_train = mask_train.reshape(train_data.shape[0], -1)
    mask_val = mask_val.reshape(val_data.shape[0], -1)

Iterations per epoch: 7318


In [11]:
import sys
pth = (os.path.abspath(os.path.dirname(os.path.dirname(os.path.dirname("__file__")))))

print(pth)

sys.path.append(pth)
from llamaTokenizer import LLaMAtokenizer

tokenizer_path = os.path.join(os.path.dirname(pth), "cptData/lit-llama/tokenizer.model")

train_frac = 0.9
seq_len = 2048
dataset = 'instruct'

tokenizer = LLaMAtokenizer(model_path=tokenizer_path)
enc = lambda s: tokenizer.encode(s, bos=False, eos=True)
dec = lambda s: tokenizer.decode(s)

/home/li/basu_workspace/nanoGPT


In [35]:
(811240*0.1)/val_data.shape[0]

3.1176357557357517

In [ ]:
import numpy as np
import sys
import os
import json
from tqdm import trange

pth = os.path.dirname(os.path.dirname(os.path.abspath(os.path.dirname(os.path.dirname(os.path.dirname(__file__))))))

###################################################################################################                          

## alpaca cleaned
with open(os.path.join(pth, 'data/alpaca/alpaca_data_cleaned.json')) as f:
    data = json.load(f)
    
data_cleaned  = ["###User: " + instruct['input'] + "\n" + instruct['instruction'] + "\n###Bot: " + instruct['output'] for instruct in data]

print("Number of examples in alpaca: ", len(data_cleaned))

## dolly
with open(os.path.join(pth, 'data/dolly/databricks-dolly-15k.jsonl')) as f:
    data = f.readlines()
    
data = [json.loads(line) for line in data]
data_cleaned_dolly  = ["###User: " + instruct['context'] + "\n" +  instruct['instruction'] + "\n###Bot: " + instruct['response'] for instruct in data]

print("Number of examples in dolly: ", len(data_cleaned_dolly))

data_cleaned += data_cleaned_dolly
############################################################################################################

sys.path.append(pth)
from llamaTokenizer import LLaMAtokenizer

tokenizer_path = os.path.join(os.path.dirname(pth), "cptData/lit-llama/tokenizer.model")

train_frac = 0.9
seq_len = 2048
dataset = 'instruct'

tokenizer = LLaMAtokenizer(model_path=tokenizer_path)
enc = lambda s: tokenizer.encode(s, bos=False, eos=True)
dec = lambda s: tokenizer.decode(s)

encoded = [enc(data_cleaned[i]) for i in trange(len(data_cleaned))]
assert len (encoded) == len(data_cleaned)

encoded = [encoded[i] for i in range(len(encoded)) if len(encoded[i]) < seq_len]

comb = np.ones((len(encoded), seq_len), dtype=np.int32)*2 ## pad with eos_token_id
j, k = 0, 0
sen_lens,l = [], []

for i in encoded:
    if k + len(i) > seq_len:
        sen_lens.append(l)
        l = [len(i)]
        j+=1
        k=len(i)
    else:
        k+=len(i)
        l.append(len(i))
    comb[j, k-len(i):k] = i

for i in range(len(comb)):
    if np.all(comb[i] == 2):
        num_sen = i-1
        break

comb = comb[:num_sen]

max_len = max([len(i) for i in sen_lens])

sen_lens = [i + [0]*(max_len - len(i)) for i in sen_lens]

inp_shape_train = sen_lens[:int(train_frac*len(comb))]
inp_shape_val = sen_lens[int(train_frac*len(comb)):]

train_ids = comb[:int(train_frac*len(comb))]
val_ids = comb[int(train_frac*len(comb)):]

assert train_ids.shape[0] == len(inp_shape_train)
assert val_ids.shape[0] == len(inp_shape_val)

print("train_ids.shape: ", train_ids.shape)
print("val_ids.shape: ", val_ids.shape)

inp_shape_train = [item for sublist in inp_shape_train for item in sublist]
inp_shape_val = [item for sublist in inp_shape_val for item in sublist]

# export to bin files
train_ids = np.array(train_ids, dtype=np.uint16)
val_ids = np.array(val_ids, dtype=np.uint16)
inp_shape_train = np.array(inp_shape_train, dtype=np.uint16)
inp_shape_val = np.array(inp_shape_val, dtype=np.uint16)

train_ids.tofile(os.path.join(pth, 'data/{dataset}/train.bin'.format(dataset=dataset)))
val_ids.tofile(os.path.join(pth, 'data/{dataset}/val.bin'.format(dataset=dataset)))
inp_shape_train.tofile(os.path.join(pth, 'data/{dataset}/inp_shape_train.bin'.format(dataset=dataset)))
inp_shape_val.tofile(os.path.join(pth, 'data/{dataset}/inp_shape_val.bin'.format(dataset=dataset)))

In [2]:
import os
import time
import math
import pickle
from contextlib import nullcontext
import numpy as np
from tqdm import trange
import torch
import torch.nn.functional as F
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.distributed import init_process_group, destroy_process_group
from utils import load_model, Sampler, get_batch, configure_optimizers, time_gpu, get_pred_idxs

# -----------------------------------------------------------------------------
# default config values designed to train a gpt2 (124M) on OpenWebText
# I/O
out_dir = 'out'
eval_interval = 2000
log_interval = 1
eval_iters = 200
eval_only = False # if True, script exits right after the first eval
sample_start = "The king exclaimed thou"
max_new_tokens=100
always_save_checkpoint = True # if True, always save a checkpoint after each eval
init_from = 'scratch' # or 'resume' or 'gpt2-medium' or 'gpt2-large' or 'gpt2-xl' or 'eval_llama' or 'llama'
# wandb logging
wandb_log = False # disabled by default
wandb_project = "transformers"
wandb_run_name = 'gpt2' # 'run' + str(time.time())
# data
dataset = 'openwebtext'
gradient_accumulation_steps = 1 # used to simulate larger batch sizes
batch_size = 32 # if gradient_accumulation_steps > 1, this is the micro-batch size
block_size = 2048
# model
n_layers = 12
n_heads = 12
n_embd = 768
dropout = 0.0 # for pretraining 0 is good, for finetuning try 0.1+
bias = False # do we use bias inside LayerNorm and Linear layers?
# adamw optimizer
learning_rate = 6e-4 # max learning rate
max_iters = 600000 # total number of training iterations
weight_decay = 1e-1
beta1 = 0.9
beta2 = 0.95
grad_clip = 1.0 # clip gradients at this value, or disable if == 0.0
# learning rate decay settings
decay_lr = True # whether to decay the learning rate
warmup_iters = 2000 # how many steps to warm up for
lr_decay_iters = 600000 # should be ~= max_iters per Chinchilla
min_lr = 6e-5 # minimum learning rate, should be ~= learning_rate/10 per Chinchilla
# DDP settings
backend = 'nccl' # 'nccl', 'gloo', etc.
# system
device = 'cuda' # examples: 'cpu', 'cuda', 'cuda:0', 'cuda:1' etc., or try 'mps' on macbooks
dtype = 'bfloat16' # 'float32', 'bfloat16', or 'float16', the latter will auto implement a GradScaler
compile = True # use PyTorch 2.0 to compile the model to be faster

ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
torch.set_default_dtype(ptdtype)

# learning block
learning_block = False
influence = 0.5

## instruct
data_type = None
break_at_eos=False
eos_token_id=1
train_on_user_only = False
# -----------------------------------------------------------------------------

config_keys = [k for k,v in globals().items() if not k.startswith('_') and isinstance(v, (int, float, bool, str))]
# exec(open('configurator.py').read()) # overrides from command line or config file

import time

eval_interval = 5
eval_iters = 40
wandb_log = True # feel free to turn on
wandb_project = 'learning-block'

sample_start = "User: Write a few words on Einstein.\nBot:"
max_new_tokens = 100

wandb_run_name = 'lb2_llama_dolly' + '_' + time.strftime("%m%d-%H%M") ## train_type,  model , dataset
dataset = 'dolly'
init_from = 'llama'

data_type = 'instruct'
out_dir = '../cptData/out/' + wandb_run_name 

# only save checkpoints if the validation loss improves
always_save_checkpoint = False

# the number of examples per iter:
# 1 batch_size * 32 grad_accum * 1024 tokens = 32,768 tokens/iter
# shakespeare has 301,966 tokens, so 1 epoch ~= 9.2 iters
batch_size = 1
gradient_accumulation_steps = 32
max_iters = 100

learning_block = True

learning_rate = 3e-4
lr_decay_iters = max_iters
decay_lr = True
warmup_iters = max_iters // 10

compile = False

break_at_eos = False
eos_token_id = 2

train_on_user_only = True


config = {k: globals()[k] for k in config_keys} # will be useful for logging
# -----------------------------------------------------------------------------

model_type = 'llama' if 'llama' in init_from else 'gpt2'

## if torch version < 2 set compile to False
if torch.__version__[0] == '1' and compile:
    print("PyTorch version < 2.0, disabling compilation")
    compile = False

print("Using Learning Block", learning_block)

# various inits, derived attributes, I/O setup
ddp = int(os.environ.get('RANK', -1)) != -1 # is this a ddp run?
if ddp:
    init_process_group(backend=backend)
    ddp_rank = int(os.environ['RANK'])
    ddp_local_rank = int(os.environ['LOCAL_RANK'])
    ddp_world_size = int(os.environ['WORLD_SIZE'])
    device = f'cuda:{ddp_local_rank}'
    torch.cuda.set_device(device)
    master_process = ddp_rank == 0 # this process will do logging, checkpointing etc.
    seed_offset = ddp_rank # each process gets a different seed
    assert gradient_accumulation_steps % torch.cuda.device_count() == 0
    gradient_accumulation_steps //= torch.cuda.device_count()
else:
    # if not ddp, we are running on a single gpu, and one process
    master_process = True
    seed_offset = 0
    ddp_world_size = 1
tokens_per_iter = gradient_accumulation_steps * ddp_world_size * batch_size * block_size

if master_process:
    os.makedirs(out_dir, exist_ok=True)
torch.manual_seed(1337 + seed_offset)
torch.backends.cuda.matmul.allow_tf32 = True # allow tf32 on matmul
torch.backends.cudnn.allow_tf32 = True # allow tf32 on cudnn
device_type = 'cuda' if 'cuda' in device else 'cpu' # for later use in torch.autocast
# note: float16 data type will automatically use a GradScaler
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)

# poor man's data loader
data_dir = os.path.join(os.path.dirname(os.path.realpath("__file__")), 'data', dataset)
train_data = np.memmap(os.path.join(data_dir, 'train.bin'), dtype=np.uint16, mode='r')
val_data = np.memmap(os.path.join(data_dir, 'val.bin'), dtype=np.uint16, mode='r')

print("Iterations per epoch:", train_data.shape[0] // tokens_per_iter)


if dataset == 'dolly':
    mask_train = np.memmap(os.path.join(data_dir, 'inp_shape_train.bin'), dtype=np.uint16, mode='r')
    mask_val = np.memmap(os.path.join(data_dir, 'inp_shape_val.bin'), dtype=np.uint16, mode='r')
    train_data = train_data.reshape(-1, block_size)
    val_data = val_data.reshape(-1, block_size)
    mask_train = mask_train.reshape(train_data.shape[0], -1)
    mask_val = mask_val.reshape(val_data.shape[0], -1)

# init these up here, can override if init_from='resume' (i.e. from a checkpoint)
iter_num = 0
best_val_loss = 1e9

# attempt to derive vocab_size from the dataset
meta_path = os.path.join(data_dir, 'meta.pkl')
meta_vocab_size = None
if os.path.exists(meta_path):
    with open(meta_path, 'rb') as f:
        meta = pickle.load(f)
    meta_vocab_size = meta['vocab_size']
    print(f"found vocab_size = {meta_vocab_size} (inside {meta_path})")

# model init
model_args = dict(n_layers=n_layers, n_heads=n_heads, n_embd=n_embd, block_size=block_size,
                  bias=bias, vocab_size=None, dropout=dropout, learning_block=learning_block) # start with model_args from command line


model, model_args = load_model(model_type, out_dir, device, learning_block, influence, init_from)

with time_gpu(device, 'model to GPU'):
    model.to(device)

## set requires grad to false for all layers except learning block

if learning_block:
    print("setting requires_grad=False for all layers except learning block")
    for name, param in model.named_parameters():
        if param.requires_grad:
            if "learning_block" not in name:
                param.requires_grad = False

## total number of parameters that requires grad
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("total params with requires grad", total_params/1e6, "M")

# initialize a GradScaler. If enabled=False scaler is a no-op
scaler = torch.cuda.amp.GradScaler(enabled=(dtype == 'float16'))

# optimizer
optimizer = configure_optimizers(model, weight_decay, learning_rate, (beta1, beta2), device_type)
# if init_from == 'resume':
#     optimizer.load_state_dict(checkpoint['optimizer'])
checkpoint = None # free up memory

# compile the model
if compile:
    with time_gpu(device, 'compiling model'):
        print("compiling the model... (takes a ~minute)")
        model = torch.compile(model) # requires PyTorch 2.0

# wrap model into DDP container
if ddp:
    model = DDP(model, device_ids=[ddp_local_rank])

sampler = Sampler(model_name = model_type, start = sample_start, device = device)

Using Learning Block True
Iterations per epoch: 25
Creating and loading model
Initializing from OG weights: /home/li/basu_workspace/nanoGPT/../cptData/lit-llama/7B/lit-llama.pth
Creating model 111.51918911933899
GPU memory used: 0.0
  Total  | Reserved |Allocated|   Max   
 79.21GB |  0.0GB  |  0.0GB  |  0.0GB  
Loading state dict 18.009955883026123
GPU memory used: 0.0
  Total  | Reserved |Allocated|   Max   
 79.21GB |  0.0GB  |  0.0GB  |  0.0GB  
Total time to load model:  129.5314872264862
model to GPU 11.540282011032104
GPU memory used: 13.6259765625
  Total  | Reserved |Allocated|   Max   
 79.21GB | 12.81GB |  12.8GB  |  12.8GB  
setting requires_grad=False for all layers except learning block
total params with requires grad 134.365184 M
num decayed parameter tensors: 64, with 134,217,728 parameters
num non-decayed parameter tensors: 64, with 147,456 parameters
using fused AdamW: True


In [4]:
wandb_log  = False

In [26]:
torch.tensor(pred_idxs, device=device).unsqueeze(2).repeat(1,1,logits.size(-1)).shape

torch.Size([1, 868, 32000])

In [29]:
logits[0][26]

tensor([ -5.9688, -12.4375,   4.2812,  ...,  -3.2031,  -2.6562,  -4.0000],
       device='cuda:0', grad_fn=<SelectBackward0>)

In [27]:
logits.gather(1, torch.tensor(pred_idxs, device=device).unsqueeze(2).repeat(1,1,logits.size(-1))).squeeze(2)

tensor([[[ -5.9688, -12.4375,   4.2812,  ...,  -3.2031,  -2.6562,  -4.0000],
         [ -5.8125,  -7.8750,   4.8125,  ...,  -3.8125,  -1.5312,  -3.0781],
         [ -2.5781,  -5.9375,   9.7500,  ...,  -1.2031,  -1.2422,  -1.2969],
         ...,
         [ -1.8750,  -2.7812,  14.8125,  ...,   0.9297,  -1.5312,  -3.6719],
         [ -3.4375, -18.5000,   4.0938,  ...,   2.5469,  -0.0437,   0.8789],
         [ -3.7969,  -5.3750,   8.6875,  ...,   0.2402,   0.4258,  -1.3047]]],
       device='cuda:0', grad_fn=<SqueezeBackward1>)

In [23]:
lgts

tensor([[[ -9.3125,  -9.3125,  -9.3125,  ...,  -9.3125,  -9.3125,  -9.3125],
         [-14.3125, -14.3125, -14.3125,  ..., -14.3125, -14.3125, -14.3125],
         [ -2.4531,  -2.4531,  -2.4531,  ...,  -2.4531,  -2.4531,  -2.4531],
         ...,
         [  2.6719,   2.6719,   2.6719,  ...,   2.6719,   2.6719,   2.6719],
         [  5.3438,   5.3438,   5.3438,  ...,   5.3438,   5.3438,   5.3438],
         [  4.3438,   4.3438,   4.3438,  ...,   4.3438,   4.3438,   4.3438]]],
       device='cuda:0', grad_fn=<SqueezeBackward1>)

In [30]:
# helps estimate an arbitrarily accurate loss over either split using many batches
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()

    print("Sampling from trained model")
    sampler.generate(model, max_new_tokens=max_new_tokens, break_at_eos = break_at_eos,eos_token_id = eos_token_id) 

    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y, mask, pred_idxs = get_batch(split, block_size, batch_size, device_type, device, train_data, val_data, 
                                   data_type = data_type, mask_train = mask_train, mask_val = mask_val, train_on_user_only = train_on_user_only)
                
            with ctx:
                logits = model(X, mask = mask)
                            
            if train_on_user_only:
                logits = logits.gather(1, torch.tensor(pred_idxs, device=device).unsqueeze(2).repeat(1,1,logits.size(-1))).squeeze(2)
                Y = Y.gather(1, torch.tensor(pred_idxs, device=device)).squeeze(1)
                
                # print(logits.shape, Y.shape)

            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), Y.view(-1), ignore_index=-1)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# learning rate decay scheduler (cosine with warmup)
def get_lr(it):
    # 1) linear warmup for warmup_iters steps
    if it < warmup_iters:
        return learning_rate * it / warmup_iters
    # 2) if it > lr_decay_iters, return min learning rate
    if it > lr_decay_iters:
        return min_lr
    # 3) in between, use cosine decay down to min learning rate
    decay_ratio = (it - warmup_iters) / (lr_decay_iters - warmup_iters)
    assert 0 <= decay_ratio <= 1
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio)) # coeff ranges 0..1
    return min_lr + coeff * (learning_rate - min_lr)

# logging
if wandb_log and master_process:
    import wandb
    wandb_api_key = "84742742b66deb0de22b5dfec52ec1f23a539d9b"
    wandb.init(project=wandb_project, name=wandb_run_name, entity='basujindal123',config=config)

# training loop
X, Y, mask, pred_idxs = get_batch('train', block_size, batch_size, device_type, device, train_data, val_data,
                       data_type = data_type, mask_train = mask_train, mask_val = mask_val, train_on_user_only = train_on_user_only)

t0 = time.time()
local_iter_num = 0 # number of iterations in the lifetime of this process
raw_model = model.module if ddp else model # unwrap DDP container if needed
iter_num_resume = iter_num

print("Training")
for iter_num in range(iter_num_resume, max_iters+1):

    # determine and set the learning rate for this iteration
    lr = get_lr(iter_num) if decay_lr else learning_rate
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    # evaluate the loss on train/val sets and write checkpoints
    if iter_num % eval_interval == 0 and master_process:
        
        with time_gpu(device,'Ealuate'):
            losses = estimate_loss()
            
        print(f"step {iter_num}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        if wandb_log:
            wandb.log({
                "iter": iter_num,
                "train/loss": losses['train'],
                "val/loss": losses['val'],
                "lr": lr,
            })
        if losses['val'] < best_val_loss or always_save_checkpoint:
            best_val_loss = losses['val']
            if iter_num > 0:
                checkpoint = {
                    'model': raw_model.state_dict(),
                    # 'optimizer': optimizer.state_dict(),
                    'model_args': model_args,
                    'iter_num': iter_num,
                    'best_val_loss': best_val_loss,
                    'config': config,
                }
                print(f"saving checkpoint to {out_dir}")
                torch.save(checkpoint, os.path.join(out_dir,'ckpt.pt'))
    if iter_num == 0 and eval_only:
        break

    with time_gpu(device, 'Training Step'):
        
        for micro_step in range(gradient_accumulation_steps):
            if ddp:
                model.require_backward_grad_sync = (micro_step == gradient_accumulation_steps - 1)
            with ctx:
                logits = model(X, mask = mask)
                
            if train_on_user_only:
                lgts = logits.gather(1, torch.tensor(pred_idxs, device=device).unsqueeze(2).repeat(1,1,logits.size(-1))).squeeze(2)
                Y = Y.gather(1, torch.tensor(pred_idxs, device=device)).squeeze(1)
                
                # print(lgts.shape, Y.shape)

            loss = F.cross_entropy(lgts.view(-1, lgts.size(-1)), Y.view(-1), ignore_index=-1)
            loss = loss / gradient_accumulation_steps # scale the loss to account for gradient accumulation
                
            # immediately async prefetch next batch while model is doing the forward pass on the GPU
            X, Y, mask, pred_idxs = get_batch('train', block_size, batch_size, device_type, device, train_data, val_data,
                                   data_type = data_type, mask_train = mask_train, mask_val = mask_val, train_on_user_only = train_on_user_only)
            
            scaler.scale(loss).backward()
        # clip the gradient
        if grad_clip != 0.0:
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)

        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad(set_to_none=True)

    # timing and logging
    t1 = time.time()
    dt = t1 - t0
    t0 = t1
    if iter_num % log_interval == 0 and master_process:
        # get loss as float. note: this is a CPU-GPU sync point
        # scale up to undo the division above, approximating the true total loss (exact would have been a sum)
        lossf = loss.item() * gradient_accumulation_steps
        print(f"iter {iter_num}: loss {lossf:.4f}, time {dt*1000:.2f}ms")
    iter_num += 1
    local_iter_num += 1

if ddp:
    destroy_process_group()

Training
Training Step 23.44026803970337
GPU memory used: 1.333984375
  Total  | Reserved |Allocated|   Max   
 79.21GB | 50.04GB | 31.13GB |  48.2GB  
iter 4: loss 1.8281, time 23531.59ms
Sampling from trained model
User: Write a few words on Einstein.
Bot: Shauna Sampath has been at the forefront of social entrepreneurism for the last decade. The founder of Latticework, Shauna Sampath is on a mission to build a 21st century infrastructure that supports the poor and disadvantaged. The Shauna Sampath organization is also set up to carry this mission. As one of the 2005 100 most influential people in the world that
---------------
User: Write a few words on Einstein.
Bot: 3rd August and the 4th of July 1776 was the day of the Boston Tea party.
It was an incident that marked the start of the American Revolution.
Labels: 1776, boston tea party, december, december, First Day of Christmas, King George, mother of George, second day of christmas, The Boston Tea Party
Susannah in 1696 is credi

KeyboardInterrupt: 

In [3]:
while True:
    ## take input
    print("Enter a senatence to continue:")
    start = str(input())
    start = "###User: " + start + ".\n###Bot:"
    print([start])

Enter a senatence to continue:
['User: Hi.\nBot:']
Enter a senatence to continue:
['User: .\nBot:']
Enter a senatence to continue:


KeyboardInterrupt: Interrupted by user

In [7]:

import random

# Generate random integer data
data = [random.randint(0, 10) for _ in range(100)]

# Randomly select sep and sen
sep = random.sample(data, 5)
sen = random.sample(data, 20)

def find_sub_list(sl,l):
    results=[]
    sll=len(sl)
    for ind in (i for i,e in enumerate(l) if e==sl[0]):
        if l[ind:ind+sll]==sl:
            results.append((ind,ind+sll-1))

    return results


# Original get_pred_idxs function
def get_pred_idxs(sen):
    sep = encode("\nBot: ")[1:-2]
    idxs = find_sub_list(sep, sen)

    pred_idxs = []
    for i in range(len(idxs)):
        for j in range(idxs[i][1], len(sen)):
            if val_data[0][j] == 2:
                pred_idxs += [sen[p] for p in range(idxs[i][1] +1, j+1)]
                break

    return pred_idxs

# Optimized get_pred_idxs function
def get_pred_idxs_optimized(sen, sep):
    idxs = find_sub_list(sep, sen)

    pred_idxs = []
    val_data_0 = val_data[0]  # Assuming val_data is defined outside the function
    for start, end in idxs:
        for j in range(start + 1, len(sen)):
            try:
                idx = val_data_0.index(2, j)
                pred_idxs.extend(sen[p] for p in range(start + 1, idx + 1))
                break
            except ValueError:
                break

    return pred_idxs

# Run the functions multiple times and calculate average duration
num_iterations = 50  # Number of iterations to run the functions

# Timing the original version
original_durations = []
for _ in range(num_iterations):
    start_time = time.time()
    original_result = get_pred_idxs(sen)
    original_durations.append(time.time() - start_time)

average_original_duration = sum(original_durations) / num_iterations

# Timing the optimized version
optimized_durations = []
for _ in range(num_iterations):
    start_time = time.time()
    optimized_result = get_pred_idxs_optimized(sen, sep)
    optimized_durations.append(time.time() - start_time)

average_optimized_duration = sum(optimized_durations) / num_iterations

print("Average original version duration:", average_original_duration)
print("Average optimized version duration:", average_optimized_duration)

Average original version duration: 1.5621185302734374e-05
Average optimized version duration: 8.759498596191407e-06


In [1]:
import os
import time
import math
import pickle
from contextlib import nullcontext
import numpy as np
from tqdm import trange
import torch
import torch.nn.functional as F
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.distributed import init_process_group, destroy_process_group
from utils import load_model, Sampler, get_batch, configure_optimizers, time_gpu, get_pred_idxs

# -----------------------------------------------------------------------------
# default config values designed to train a gpt2 (124M) on OpenWebText
# I/O
out_dir = 'out'
eval_interval = 2000
log_interval = 1
eval_iters = 200
eval_only = False # if True, script exits right after the first eval
sample_start = "The king exclaimed thou"
max_new_tokens=100
always_save_checkpoint = True # if True, always save a checkpoint after each eval
init_from = 'scratch' # or 'resume' or 'gpt2-medium' or 'gpt2-large' or 'gpt2-xl' or 'eval_llama' or 'llama'
# wandb logging
wandb_log = False # disabled by default
wandb_project = "transformers"
wandb_run_name = 'gpt2' # 'run' + str(time.time())
# data
dataset = 'openwebtext'
gradient_accumulation_steps = 1 # used to simulate larger batch sizes
batch_size = 32 # if gradient_accumulation_steps > 1, this is the micro-batch size
block_size = 2048
# model
n_layers = 12
n_heads = 12
n_embd = 768
dropout = 0.0 # for pretraining 0 is good, for finetuning try 0.1+
bias = False # do we use bias inside LayerNorm and Linear layers?
# adamw optimizer
learning_rate = 6e-4 # max learning rate
max_iters = 600000 # total number of training iterations
weight_decay = 1e-1
beta1 = 0.9
beta2 = 0.95
grad_clip = 1.0 # clip gradients at this value, or disable if == 0.0
# learning rate decay settings
decay_lr = True # whether to decay the learning rate
warmup_iters = 2000 # how many steps to warm up for
lr_decay_iters = 600000 # should be ~= max_iters per Chinchilla
min_lr = 6e-5 # minimum learning rate, should be ~= learning_rate/10 per Chinchilla
# DDP settings
backend = 'nccl' # 'nccl', 'gloo', etc.
# system
device = 'cuda' # examples: 'cpu', 'cuda', 'cuda:0', 'cuda:1' etc., or try 'mps' on macbooks
dtype = 'bfloat16' # 'float32', 'bfloat16', or 'float16', the latter will auto implement a GradScaler
compile = True # use PyTorch 2.0 to compile the model to be faster

ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
torch.set_default_dtype(ptdtype)

# learning block
learning_block = False
influence = 0.5

## instruct
data_type = None
break_at_eos=False
eos_token_id=1
# -----------------------------------------------------------------------------

config_keys = [k for k,v in globals().items() if not k.startswith('_') and isinstance(v, (int, float, bool, str))]
# exec(open('configurator.py').read()) # overrides from command line or config file
import time

eval_interval = 5
eval_iters = 40
wandb_log = True # feel free to turn on
wandb_project = 'learning-block'

sample_start = "User: Write a few words on Einstein.\nBot:"
max_new_tokens = 100

wandb_run_name = 'lb2_llama_dolly' + '_' + time.strftime("%m%d-%H%M") ## train_type,  model , dataset
dataset = 'dolly'
init_from = 'llama'

data_type = 'instruct'
out_dir = '../cptData/out/' + wandb_run_name 

# only save checkpoints if the validation loss improves
always_save_checkpoint = False

# the number of examples per iter:
# 1 batch_size * 32 grad_accum * 1024 tokens = 32,768 tokens/iter
# shakespeare has 301,966 tokens, so 1 epoch ~= 9.2 iters
batch_size = 1
gradient_accumulation_steps = 32
max_iters = 500

learning_block = True

learning_rate = 3e-4
lr_decay_iters = max_iters
decay_lr = True
warmup_iters = max_iters // 10

compile = False

break_at_eos = True
eos_token_id = 2
config = {k: globals()[k] for k in config_keys} # will be useful for logging
# -----------------------------------------------------------------------------

model_type = 'llama' if 'llama' in init_from else 'gpt2'

## if torch version < 2 set compile to False
if torch.__version__[0] == '1' and compile:
    print("PyTorch version < 2.0, disabling compilation")
    compile = False

print("Using Learning Block", learning_block)

# various inits, derived attributes, I/O setup
ddp = int(os.environ.get('RANK', -1)) != -1 # is this a ddp run?
if ddp:
    init_process_group(backend=backend)
    ddp_rank = int(os.environ['RANK'])
    ddp_local_rank = int(os.environ['LOCAL_RANK'])
    ddp_world_size = int(os.environ['WORLD_SIZE'])
    device = f'cuda:{ddp_local_rank}'
    torch.cuda.set_device(device)
    master_process = ddp_rank == 0 # this process will do logging, checkpointing etc.
    seed_offset = ddp_rank # each process gets a different seed
    assert gradient_accumulation_steps % torch.cuda.device_count() == 0
    gradient_accumulation_steps //= torch.cuda.device_count()
else:
    # if not ddp, we are running on a single gpu, and one process
    master_process = True
    seed_offset = 0
    ddp_world_size = 1
tokens_per_iter = gradient_accumulation_steps * ddp_world_size * batch_size * block_size

if master_process:
    os.makedirs(out_dir, exist_ok=True)
torch.manual_seed(1337 + seed_offset)
torch.backends.cuda.matmul.allow_tf32 = True # allow tf32 on matmul
torch.backends.cudnn.allow_tf32 = True # allow tf32 on cudnn
device_type = 'cuda' if 'cuda' in device else 'cpu' # for later use in torch.autocast
# note: float16 data type will automatically use a GradScaler
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)

# poor man's data loader
data_dir = os.path.join(os.path.dirname(os.path.realpath("__file__")), 'data', dataset)
train_data = np.memmap(os.path.join(data_dir, 'train.bin'), dtype=np.uint16, mode='r')
val_data = np.memmap(os.path.join(data_dir, 'val.bin'), dtype=np.uint16, mode='r')

print("Iterations per epoch:", train_data.shape[0] // tokens_per_iter)


if dataset == 'dolly':
    mask_train = np.memmap(os.path.join(data_dir, 'inp_shape_train.bin'), dtype=np.uint16, mode='r')
    mask_val = np.memmap(os.path.join(data_dir, 'inp_shape_val.bin'), dtype=np.uint16, mode='r')
    train_data = train_data.reshape(-1, block_size)
    val_data = val_data.reshape(-1, block_size)
    mask_train = mask_train.reshape(train_data.shape[0], -1)
    mask_val = mask_val.reshape(val_data.shape[0], -1)

train_on_user_only = True
# training loop
X, Y, mask, pred_idxs = get_batch('train', block_size, batch_size, device_type, device, train_data, val_data,
                       data_type = data_type, mask_train = mask_train, mask_val = mask_val, train_on_user_only = train_on_user_only)

Using Learning Block True
Iterations per epoch: 25


In [30]:
decode(X.tolist())

['User: What is the difference between a coastal cruising and blue water sailboat?\nBot: The designs, accommodations and compromises for these boats can be very different and reflect the likely sea conditions these boats would encounter, expected time at sea and different experiences of passenger comfort.  The primary difference between these boats begins with the shape of the hull since many other design choices tend to be dictated by hull shape.\n\nCoastal cruisers, which are generally sailed near shore and in safer weather conditions, \ntend to have a flatter hull shapes since these boats, which means they tend to "slap" the water when coming off a wave.  With a flatter hull shape, boat architects can make greater use of the cabin space and so coastal cruisers will generally have a more spacious interior.  The helm (or the outdoor area from which the boat is navigated) is generally less protected and generally speaking, offers greater comfort and ease of moving around.  Safety, prot

In [2]:
from utils import load_model, get_tokenizer, print_gpu_utilization, time_gpu

encode, decode = get_tokenizer(model_type)

a = Y.gather(1, torch.tensor(pred_idxs, device=device)).squeeze(1)

decode(a.tolist()[0])

'The designs, accommodations and compromises for these boats can be very different and reflect the likely sea conditions these boats would encounter, expected time at sea and different experiences of passenger comfort.  The primary difference between these boats begins with the shape of the hull since many other design choices tend to be dictated by hull shape.\n\nCoastal cruisers, which are generally sailed near shore and in safer weather conditions, \ntend to have a flatter hull shapes since these boats, which means they tend to "slap" the water when coming off a wave.  With a flatter hull shape, boat architects can make greater use of the cabin space and so coastal cruisers will generally have a more spacious interior.  The helm (or the outdoor area from which the boat is navigated) is generally less protected and generally speaking, offers greater comfort and ease of moving around.  Safety, protection and expected time away from marinas mean these boats can refill fresh water, rech

In [29]:
"""
Sample from a trained model
"""

from contextlib import nullcontext
import torch
from utils import load_model, get_tokenizer, print_gpu_utilization, time_gpu

# -----------------------------------------------------------------------------
init_from = ['resume', 'eval_llama', 'llama', 'gpt2-small', 'gpt2-medium', 'gpt2-large', 'gpt2-xl'][1] # or 'resume' or 'gpt2-medium' or 'gpt2-large' or 'gpt2-xl'
out_dir = "/home/li/basu_workspace/cptData/out/lb2_llama_dolly_0605-0501/ckpt.pt"
start = "User: Capital of France?\n"  # or "<|endoftext|>" or etc. Can also specify a file, use as: "FILE:prompt.txt"
num_samples =  3  # number of samples to draw
max_new_tokens = 200 # number of tokens generated in each sample
temperature = 0.8 # 1.0 = no change, < 1.0 = less random, > 1.0 = more random, in predictions
top_k = 200 # retain only the top_k most likely tokens, clamp others to have 0 probability
seed = 1337
device = 'cuda' # examples: 'cpu', 'cuda', 'cuda:0', 'cuda:1', etc.
dtype = 'bfloat16' # 'float32' or 'bfloat16' or 'float16'
compile = False # use PyTorch 2.0 to compile the model to be faster

# learning block
learning_block = True
influence = 0.5
# -----------------------------------------------------------------------------

model_type = 'llama' if 'llama' in init_from else 'gpt2'

# sampling = "continuous"
sampling = "discrete"

# exec(open('configurator.py').read()) # overrides from command line or config file
# -----------------------------------------------------------------------------

torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cuda.matmul.allow_tf32 = True # allow tf32 on matmul
torch.backends.cudnn.allow_tf32 = True # allow tf32 on cudnn
device_type = 'cuda' if 'cuda' in device else 'cpu' # for later use in torch.autocast
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]

torch.set_default_dtype(ptdtype)

ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)


# # model
# model, model_args = load_model(model_type, out_dir, device, learning_block, influence, init_from)

# model.eval()

# with time_gpu('move model to device'):
#     model.to(device)

# print_gpu_utilization()

# if compile:
#     model = torch.compile(model) # requires PyTorch 2.0 (optional)


# tokenizer
encode, decode = get_tokenizer(model_type)                 


In [5]:
import numpy as np
import os

data_dir = "/home/li/basu_workspace/nanoGPT/data/dolly"
val_data = np.memmap(os.path.join(data_dir, 'val.bin'), dtype=np.uint16, mode='r')
val_data = val_data.reshape(-1, 2048)

In [ ]:
val_data[0].tolist()[get_pred_idxs(val_data[0].tolist())]

In [54]:
def find_sub_list(sl,l):
    results=[]
    sll=len(sl)
    print(sll)
    for ind in (i for i,e in enumerate(l) if e==sl[0]):
        if l[ind:ind+sll]==sl:
            results.append((ind,ind+sll-1))

    return results

def get_pred_idxs(sen):
    sep = encode("\nBot: ")[1:-2]
    print(sep)
    idxs = find_sub_list(sep, sen)
    print("idxs", idxs)

    pred_idxs = []
    for i in range(len(idxs)):
        for j in range(idxs[i][1], len(sen)):
            if val_data[0][j] == 2:
                pred_idxs += [p for p in range(idxs[i][1], j)]     
                break
            
    return pred_idxs

# sen = val_data[0].tolist() 


In [13]:
start = "User: What is the best food in the world?\nBot: "
start_ids = encode(start)
tkns = (torch.tensor(start_ids, dtype=torch.long, device=device)[None, ...])

# run generation
with torch.no_grad():
    with ctx:
        for k in range(num_samples):
            with time_gpu('generate'):
                print("generating sample", k+1, "of", num_samples)
                y = model.generate(tkns, max_new_tokens, temperature=temperature, top_k=top_k)
                print(decode(y[0].tolist()))
                print('---------------')

generating sample 1 of 3
User: What is the best food in the world?
Bot:  a F This2 alsoB is,7 byB., and can the,:,
 of D
 itsB from: to as to What to
., are. in there User
 and a., for a to that to toB
) and also following, and- you of.ot the.. aot?:2 by11 will6, on inB as and the in a
 with, and and of to0: more to, a- of: is9 not be.  of.
 an: of is The,?". also, to, the of  in? to the: is you the ( are,4 -
 of: your theing, to
 the0 " in
---------------
Time taken:  16.523746013641357
GPU memory used: 0.01953125
  Total  | Reserved |Allocated|   Max   
 79.21GB | 12.88GB | 12.81GB | 12.84GB 
generating sample 2 of 3
User: What is the best food in the world?
Bot:  the of,. each9 the1 is of are of:5,,,,3?, to by, I ining and of: the.0 into. C in9 a B to,' of6, a.4-,0, of ,,, to::,,B
2..),:., to: (, to was of,us to  is and:  to?
,
 a:B9,B of was for,ings
1 the, canB from: is,0, UserB in
,,2 of then and of. a9 in, (. ,. is-,,: usedB
:. Hot
. the one is: ofing. the such:
---------------


In [ ]:
if sampling == "discrete":
    # encode the beginning of the prompt
    if start.startswith('FILE:'):
        with open(start[5:], 'r', encoding='utf-8') as f:
            start = f.read()
            
    start_ids = encode(start)
    tkns = (torch.tensor(start_ids, dtype=torch.long, device=device)[None, ...])

    # run generation
    with torch.no_grad():
        with ctx:
            for k in range(num_samples):
                with time_gpu('generate'):
                    print("generating sample", k+1, "of", num_samples)
                    y = model.generate(tkns, max_new_tokens, temperature=temperature, top_k=top_k)
                    print(decode(y[0].tolist()))
                    print('---------------')


if sampling == "continuous":

    while True:
        ## take input
        print("Enter a sentence to continue:")
        start = str(input())
        start_ids = encode(start)
        tkns = (torch.tensor(start_ids, dtype=torch.long, device=device)[None, ...])

        # run generation
        with torch.no_grad():
            with ctx:
                for k in range(num_samples):
                    with time_gpu('Time to generate'):
                        print("Sample", k+1, "------------------------------------")
                        y = model.generate(tkns, max_new_tokens, temperature=temperature, top_k=top_k)
                        print(decode(y[0].tolist()))
                        print('---------------')

In [1]:
import os
import time
import math
import pickle
from contextlib import nullcontext
import numpy as np
import copy
import torch
from tqdm import trange
import torch.nn.functional as F
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.distributed import init_process_group, destroy_process_group

from utils import load_model, Sampler, get_batch, configure_optimizers, time_gpu

# -----------------------------------------------------------------------------
# default config values designed to train a gpt2 (124M) on OpenWebText
# I/O
out_dir = 'out'
eval_interval = 2000
log_interval = 1
eval_iters = 200
eval_only = False # if True, script exits right after the first eval
sample_start = "User: What is the capital of India.\nBot:"
always_save_checkpoint = True # if True, always save a checkpoint after each eval
init_from = 'scratch' # or 'resume' or 'gpt2-medium' or 'gpt2-large' or 'gpt2-xl' or 'eval_llama' or 'llama'
# wandb logging
wandb_log = False # disabled by default
wandb_project = "transformers"
wandb_run_name = 'gpt2' # 'run' + str(time.time())
# data
dataset = 'openwebtext'
gradient_accumulation_steps = 1 # used to simulate larger batch sizes
batch_size = 32 # if gradient_accumulation_steps > 1, this is the micro-batch size
block_size = 2048
# model
n_layers = 12
n_heads = 12
n_embd = 768
dropout = 0.0 # for pretraining 0 is good, for finetuning try 0.1+
bias = False # do we use bias inside LayerNorm and Linear layers?
# adamw optimizer
learning_rate = 6e-4 # max learning rate
max_iters = 600000 # total number of training iterations
weight_decay = 1e-1
beta1 = 0.9
beta2 = 0.95
grad_clip = 1.0 # clip gradients at this value, or disable if == 0.0
# learning rate decay settings
decay_lr = True # whether to decay the learning rate
warmup_iters = 2000 # how many steps to warm up for
lr_decay_iters = 600000 # should be ~= max_iters per Chinchilla
min_lr = 6e-5 # minimum learning rate, should be ~= learning_rate/10 per Chinchilla
# DDP settings
backend = 'nccl' # 'nccl', 'gloo', etc.
# system
device = 'cuda' # examples: 'cpu', 'cuda', 'cuda:0', 'cuda:1' etc., or try 'mps' on macbooks
dtype = 'bfloat16' # 'float32', 'bfloat16', or 'float16', the latter will auto implement a GradScaler
compile = True # use PyTorch 2.0 to compile the model to be faster
max_new_tokens = 10
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
torch.set_default_dtype(ptdtype)

# learning block
learning_block = False
influence = 0.5

## instruct
data_type = None
# -----------------------------------------------------------------------------

config_keys = [k for k,v in globals().items() if not k.startswith('_') and isinstance(v, (int, float, bool, str))]
import time

eval_interval = 5
eval_iters = 40
wandb_log = False # feel free to turn on
wandb_project = 'learning-block'


wandb_run_name = 'lb2_llama_dolly' + '_' + time.strftime("%m%d-%H%M") ## train_type,  model , dataset
dataset = 'dolly'
init_from = 'llama'

data_type = 'instruct'
out_dir = '../cptData/out/' + wandb_run_name 

# only save checkpoints if the validation loss improves
always_save_checkpoint = False

# the number of examples per iter:
# 1 batch_size * 32 grad_accum * 1024 tokens = 32,768 tokens/iter
# shakespeare has 301,966 tokens, so 1 epoch ~= 9.2 iters
batch_size = 1
gradient_accumulation_steps = 32
max_iters = 100

# finetune at constant LR
learning_rate = 3e-5
decay_lr = False
learning_block = True

compile = False

config = {k: globals()[k] for k in config_keys} # will be useful for logging
# -----------------------------------------------------------------------------

model_type = 'llama' if 'llama' in init_from else 'gpt2'

## if torch version < 2 set compile to False
if torch.__version__[0] == '1' and compile:
    print("PyTorch version < 2.0, disabling compilation")
    compile = False

print("Using Learning Block", learning_block)

# various inits, derived attributes, I/O setup
ddp = int(os.environ.get('RANK', -1)) != -1 # is this a ddp run?
if ddp:
    init_process_group(backend=backend)
    ddp_rank = int(os.environ['RANK'])
    ddp_local_rank = int(os.environ['LOCAL_RANK'])
    ddp_world_size = int(os.environ['WORLD_SIZE'])
    device = f'cuda:{ddp_local_rank}'
    torch.cuda.set_device(device)
    master_process = ddp_rank == 0 # this process will do logging, checkpointing etc.
    seed_offset = ddp_rank # each process gets a different seed
    assert gradient_accumulation_steps % torch.cuda.device_count() == 0
    gradient_accumulation_steps //= torch.cuda.device_count()
else:
    # if not ddp, we are running on a single gpu, and one process
    master_process = True
    seed_offset = 0
    ddp_world_size = 1
tokens_per_iter = gradient_accumulation_steps * ddp_world_size * batch_size * block_size

if master_process:
    os.makedirs(out_dir, exist_ok=True)
torch.manual_seed(1337 + seed_offset)
torch.backends.cuda.matmul.allow_tf32 = True # allow tf32 on matmul
torch.backends.cudnn.allow_tf32 = True # allow tf32 on cudnn
device_type = 'cuda' if 'cuda' in device else 'cpu' # for later use in torch.autocast
# note: float16 data type will automatically use a GradScaler
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)

# poor man's data loader
data_dir = os.path.join(os.path.dirname(os.path.realpath("__file__")), 'data', dataset)
train_data = np.memmap(os.path.join(data_dir, 'train.bin'), dtype=np.uint16, mode='r')
val_data = np.memmap(os.path.join(data_dir, 'val.bin'), dtype=np.uint16, mode='r')

print("Iterations per epoch:", train_data.shape[0] // tokens_per_iter)


if dataset == 'dolly':
    mask_train = np.memmap(os.path.join(data_dir, 'inp_shape_train.bin'), dtype=np.uint16, mode='r')
    mask_val = np.memmap(os.path.join(data_dir, 'inp_shape_val.bin'), dtype=np.uint16, mode='r')
    train_data = train_data.reshape(-1, block_size)
    val_data = val_data.reshape(-1, block_size)
    mask_train = mask_train.reshape(train_data.shape[0], -1)
    mask_val = mask_val.reshape(val_data.shape[0], -1)
    

# init these up here, can override if init_from='resume' (i.e. from a checkpoint)
iter_num = 0
best_val_loss = 1e9

# attempt to derive vocab_size from the dataset
meta_path = os.path.join(data_dir, 'meta.pkl')
meta_vocab_size = None
if os.path.exists(meta_path):
    with open(meta_path, 'rb') as f:
        meta = pickle.load(f)
    meta_vocab_size = meta['vocab_size']
    print(f"found vocab_size = {meta_vocab_size} (inside {meta_path})")

# model init
model_args = dict(n_layers=n_layers, n_heads=n_heads, n_embd=n_embd, block_size=block_size,
                  bias=bias, vocab_size=None, dropout=dropout, learning_block=learning_block) # start with model_args from command line


model, model_args = load_model(model_type, out_dir, device, learning_block, influence, init_from)
with time_gpu(device, 'model to GPU'):
    model.to(device)

## set requires grad to false for all layers except learning block

if learning_block:
    print("setting requires_grad=False for all layers except learning block")
    for name, param in model.named_parameters():
        if param.requires_grad:
            if "learning_block" not in name:
                param.requires_grad = False

## total number of parameters that requires grad
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("total params with requires grad", total_params/1e6, "M")

# initialize a GradScaler. If enabled=False scaler is a no-op
scaler = torch.cuda.amp.GradScaler(enabled=(dtype == 'float16'))

# optimizer
optimizer = configure_optimizers(model, weight_decay, learning_rate, (beta1, beta2), device_type)
# if init_from == 'resume':
#     optimizer.load_state_dict(checkpoint['optimizer'])
checkpoint = None # free up memory

# compile the model
if compile:
    print("compiling the model... (takes a ~minute)")
    unoptimized_model = model
    model = torch.compile(model) # requires PyTorch 2.0

# wrap model into DDP container
if ddp:
    model = DDP(model, device_ids=[ddp_local_rank])

sampler = Sampler(model_name = model_type, start = sample_start, device = device)

Using Learning Block True
Iterations per epoch: 25
Creating and loading model
Initializing from OG weights: /home/li/basu_workspace/nanoGPT/../cptData/lit-llama/7B/lit-llama.pth
Creating model 123.11571097373962
GPU memory used: 0.0
  Total  | Reserved |Allocated|   Max   
 79.21GB |  0.0GB  |  0.0GB  |  0.0GB  
Loading state dict 19.42014741897583
GPU memory used: 0.0
  Total  | Reserved |Allocated|   Max   
 79.21GB |  0.0GB  |  0.0GB  |  0.0GB  
Time to load model:  142.7067472934723


NameError: name 'copy' is not defined

In [7]:
def make_mask(inp, seq_len):
    mask = np.eye((seq_len))
    ptr = 0
    for len in inp:
        if len == 0:
            break
        mask[ptr:ptr+len, ptr:ptr+len] = np.tril(np.ones((len, len)))
        ptr += len
    return mask

def get_batch(split, block_size, batch_size, device_type, device, train_data, val_data, data_type=None, mask_train = None, mask_val = None):
    
    if data_type == None:
        data = train_data if split == 'train' else val_data
        ix = torch.randint(len(data) - block_size, (batch_size,))
        x = torch.stack([torch.from_numpy((data[i:i+block_size]).astype(np.int64)) for i in ix])
        y = torch.stack([torch.from_numpy((data[i+1:i+1+block_size]).astype(np.int64)) for i in ix])
        
        if 'cuda' in device_type:
            # pin arrays x,y, which allows us to move them to GPU asynchronously (non_blocking=True)
            x, y = x.pin_memory().to(device, non_blocking=True), y.pin_memory().to(device, non_blocking=True)
        else:
            x, y = x.to(device), y.to(device)
        return x, y, None
    
    elif data_type == 'instruct':
        
        data = train_data if split == 'train' else val_data
        mask_data = mask_train if split == 'train' else mask_val
        
        ix = torch.randint(data.shape[0] - 1, (batch_size,))
        
        x = torch.stack([torch.from_numpy((data[i][:-1]).astype(np.int64)) for i in ix])
        y = torch.stack([torch.from_numpy((data[i+1][1:]).astype(np.int64)) for i in ix])
        mask = torch.stack([torch.from_numpy(make_mask(mask_data[i], block_size)[:-1, :-1]) for i in ix])
        ## conver mask to bool
        mask = mask.bool()
        
        if 'cuda' in device_type:
            # pin arrays x,y, which allows us to move them to GPU asynchronously (non_blocking=True)
            x, y, mask = x.pin_memory().to(device, non_blocking=True), y.pin_memory().to(device, non_blocking=True), mask.pin_memory().to(device, non_blocking=True)
        else:
            x, y, mask = x.to(device), y.to(device), mask.to(device)
        return x, y, mask
    
# helps estimate an arbitrarily accurate loss over either split using many batches
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()

    print("Sampling from trained model")
    sampler.generate(model, max_new_tokens=max_new_tokens)

    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in trange(eval_iters):
            X, Y, mask = get_batch(split, block_size, batch_size, device_type, device, train_data, val_data, data_type = data_type,
                                   mask_train = mask_train, mask_val = mask_val)
            with ctx:
                logits = model(X, mask=mask)
                # print(logits)
                loss = F.cross_entropy(logits.view(-1, logits.size(-1)), Y.view(-1), ignore_index=-1)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# learning rate decay scheduler (cosine with warmup)
def get_lr(it):
    # 1) linear warmup for warmup_iters steps
    if it < warmup_iters:
        return learning_rate * it / warmup_iters
    # 2) if it > lr_decay_iters, return min learning rate
    if it > lr_decay_iters:
        return min_lr
    # 3) in between, use cosine decay down to min learning rate
    decay_ratio = (it - warmup_iters) / (lr_decay_iters - warmup_iters)
    assert 0 <= decay_ratio <= 1
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio)) # coeff ranges 0..1
    return min_lr + coeff * (learning_rate - min_lr)

# logging
if wandb_log and master_process:
    import wandb
    ## add wandb key
    wandb_api_key = "84742742b66deb0de22b5dfec52ec1f23a539d9b"
    wandb.init(project=wandb_project, name=wandb_run_name, entity='basujindal123',config=config)

# training loop
X, Y, mask = get_batch('train', block_size, batch_size, device_type, device, train_data, val_data, data_type = data_type, mask_train = mask_train, mask_val = mask_val)
t0 = time.time()
local_iter_num = 0 # number of iterations in the lifetime of this process
raw_model = model.module if ddp else model # unwrap DDP container if needed
iter_num_resume = iter_num

In [8]:
with time_gpu(device,'Ealuate'):
    losses = estimate_loss()

Sampling from trained model
User: What is the capital of India.
Bot: *This is impossible.
User: What is
---------------
User: What is the capital of India.
Bot: I am sorry, I am not a human.
---------------


100%|██████████| 40/40 [00:35<00:00,  1.13it/s]

Ealuate 66.29910826683044
GPU memory used: 17.65234375
  Total  | Reserved |Allocated|   Max   
 79.21GB | 14.61GB | 13.55GB | 14.31GB 


In [10]:
print("Training")
for iter_num in range(iter_num_resume, max_iters+1):

    # determine and set the learning rate for this iteration
    lr = get_lr(iter_num) if decay_lr else learning_rate
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    # evaluate the loss on train/val sets and write checkpoints
    if iter_num % eval_interval == 0 and master_process:
        
        with time_gpu(device,'Ealuate'):
            losses = estimate_loss()
            
        print(f"step {iter_num}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        if wandb_log:
            wandb.log({
                "iter": iter_num,
                "train/loss": losses['train'],
                "val/loss": losses['val'],
                "lr": lr,
            })
        if losses['val'] < best_val_loss or always_save_checkpoint:
            best_val_loss = losses['val']
            if iter_num > 0:
                checkpoint = {
                    'model': raw_model.state_dict(),
                    # 'optimizer': optimizer.state_dict(),
                    'model_args': model_args,
                    'iter_num': iter_num,
                    'best_val_loss': best_val_loss,
                    'config': config,
                }
                print(f"saving checkpoint to {out_dir}")
                torch.save(checkpoint, os.path.join(out_dir,'ckpt.pt'))
    if iter_num == 0 and eval_only:
        break

    with time_gpu(device, 'Training Step'):
        
        for micro_step in range(gradient_accumulation_steps):
            if ddp:
                model.require_backward_grad_sync = (micro_step == gradient_accumulation_steps - 1)
            with ctx:
                logits = model(X, mask=mask)
                ## check if any nan values in logits
                if torch.isnan(logits).any():
                    raise ValueError('NaN values in logits')
                loss = F.cross_entropy(logits.view(-1, logits.size(-1)), Y.view(-1), ignore_index=-1)
                loss = loss / gradient_accumulation_steps # scale the loss to account for gradient accumulation
                
            # immediately async prefetch next batch while model is doing the forward pass on the GPU
            X, Y, mask = get_batch('train', block_size, batch_size, device_type, device, train_data, val_data, 
                                   data_type = data_type, mask_train = mask_train, mask_val = mask_val)
            scaler.scale(loss).backward()
        # clip the gradient
        if grad_clip != 0.0:
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)

        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad(set_to_none=True)

    # timing and logging
    t1 = time.time()
    dt = t1 - t0
    t0 = t1
    if iter_num % log_interval == 0 and master_process:
        # get loss as float. note: this is a CPU-GPU sync point
        # scale up to undo the division above, approximating the true total loss (exact would have been a sum)
        lossf = loss.item() * gradient_accumulation_steps
        print(f"iter {iter_num}: loss {lossf:.4f}, time {dt*1000:.2f}ms")
    iter_num += 1
    local_iter_num += 1

if ddp:
    destroy_process_group()

Training
step 0: train loss 12.8125, val loss 12.6875
Training Step 49.16111731529236
GPU memory used: 17.142578125
  Total  | Reserved |Allocated|   Max   
 79.21GB | 49.05GB |  30.6GB  | 47.53GB 
iter 0: loss 18.4256, time 80550.46ms
Training Step 47.726683139801025
GPU memory used: 0.013671875
  Total  | Reserved |Allocated|   Max   
 79.21GB | 49.05GB |  30.6GB  | 47.54GB 
iter 1: loss 12.0520, time 47748.21ms
Training Step 48.63527154922485
GPU memory used: 4.2880859375
  Total  | Reserved |Allocated|   Max   
 79.21GB | 49.05GB |  30.6GB  | 47.54GB 
iter 2: loss 11.7840, time 48774.90ms
Training Step 47.903974771499634
GPU memory used: 4.013671875
  Total  | Reserved |Allocated|   Max   
 79.21GB | 49.05GB |  30.6GB  | 47.54GB 
iter 3: loss 10.4763, time 48111.75ms
Training Step 47.782862186431885
GPU memory used: 0.0
  Total  | Reserved |Allocated|   Max   
 79.21GB | 49.05GB |  30.6GB  | 47.54GB 
iter 4: loss 11.6493, time 47795.05ms
step 5: train loss 12.8125, val loss 12.6875

KeyboardInterrupt: 

In [38]:
import os
import requests
import tiktoken
import numpy as np
import sys
from llamaTokenizer import LLaMAtokenizer
import matplotlib.pyplot as plt

tokenizer_path = "/home/li/basu_workspace/llama/tokenizer.model"
tokenizer = LLaMAtokenizer(model_path=tokenizer_path)

enc = lambda s: tokenizer.encode(s, bos=False, eos=True)
dec = lambda s: tokenizer.decode(s)

import json

with open('/home/li/basu_workspace/nanoGPT/data/dolly/databricks-dolly-15k.jsonl') as f:
    data = f.readlines()
    
data = [json.loads(line) for line in data]
data_cleaned  = ["User: " + instruct['instruction'] + "\nBot: " + instruct['response'] for instruct in data]

encoded = []
for sentence in data_cleaned:
    encoded.append(enc(sentence))
assert len (encoded) == len(data_cleaned)

seq_len = 2048
### create new encoded if len(encoded[i]) < seq_len

encoded = [encoded[i] for i in range(len(encoded)) if len(encoded[i]) < seq_len]

In [49]:
comb = np.ones((len(encoded), seq_len), dtype=np.int32)*2
j = 0
k = 0
sen_lens = []
l = []
for i in encoded:
    if k + len(i) > seq_len:
        sen_lens.append(l)
        l = [len(i)]
        j+=1
        k=len(i)
    else:
        k+=len(i)
        l.append(len(i))
    comb[j, k-len(i):k] = i

for i in range(len(comb)):
    if np.all(comb[i] == 2):
        num_sen = i-1
        break

comb = comb[:num_sen]

max_len = max([len(i) for i in sen_lens])
## pad sen_lens with zeros
sen_lens = [i + [0]*(max_len - len(i)) for i in sen_lens]
sen_lens = [item for sublist in sen_lens for item in sublist]

train_frac = 0.9
train_ids = comb[:int(train_frac*len(comb))]
val_ids = comb[int(train_frac*len(comb)):]


# export to bin files
train_ids = np.array(train_ids, dtype=np.uint16)
val_ids = np.array(val_ids, dtype=np.uint16)
sen_lens = np.array(sen_lens, dtype=np.uint16)
train_ids.tofile('/home/li/basu_workspace/nanoGPT/data/dolly/train.bin')
val_ids.tofile( '/home/li/basu_workspace/nanoGPT/data/dolly/val.bin')
sen_lens.tofile('/home/li/basu_workspace/nanoGPT/data/dolly/sen_lens.bin')

In [4]:
import json

with open('/home/li/basu_workspace/nanoGPT/data/databricks-dolly-15k.jsonl') as f:
    data = f.readlines()
    
data = [json.loads(line) for line in data]
data_cleaned  = ["User: " + instruct['instruction'] + "\nBot: " + instruct['response'] for instruct in data]

In [5]:
encoded = []
for sentence in data_cleaned:
    encoded.append(enc(sentence))
assert len (encoded) == len(data_cleaned)

In [6]:
lens = [len(encoded[i]) for i in range(len(encoded))]
sorted_lens = sorted(lens)
pl = plt.hist(sorted_lens[:-200], bins=1000)

In [ ]:
import torch
import safetensors
from safetensors.torch import save_model

save_model(model, 'my_model.safetensors')

# tensors = {}
# with safe_open("model.safetensors", framework="pt", device="cpu") as f:
#    for key in f.keys():
#        tensors[key] = f.get_tensor(key)

In [ ]:
import safetensors
safetensors.torch.load_model(model, 'my_model.safetensors')